In [1]:
import json

import dash_bootstrap_components as dbc
import dash_cytoscape as cyto
import networkx as nx
import plotly.graph_objects as go
from dash import Dash, dcc, html
from dash.dependencies import Input, Output
from dash.exceptions import PreventUpdate
from vu_panel import Topics

%load_ext autoreload
%autoreload 2
%reload_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
external_stylesheets = [dbc.themes.CERULEAN]
topics = Topics(**json.load(open("panel_topics.json", "r")))

In [27]:
app = Dash(external_stylesheets=external_stylesheets)

app.layout = html.Div(
    [
        cyto.Cytoscape(
            id="cytoscape-compound",
            layout={
                "name": "grid",
                # "roots": "[id = 'root']",
                # "directed": True,
                # "nodeSpacing": 50
                "spacingFactor": 1.5,
            },
            style={"width": "100%", "height": "500px"},
            stylesheet=[
                {"selector": "node", "style": {"content": "data(label)"}},
                {"selector": ".countries", "style": {"width": 5}},
                {"selector": ".cities", "style": {"line-style": "dashed"}},
            ],
            elements=[
                # Root Node
                # {"data": {"id": "root", "label": "Countries"}},
                # Grand Parent Nodes
                {
                    "data": {"id": "asia", "label": "Asia"},
                },
                {
                    "data": {"id": "europe", "label": "Europe"},
                },
                {
                    "data": {"id": "africa", "label": "Africa"},
                },
                # Parent Nodes
                {
                    "data": {"id": "ch", "label": "China", "parent": "asia"},
                },
                {
                    "data": {"id": "ger", "label": "Germany", "parent": "europe"},
                },
                {
                    "data": {"id": "pk", "label": "Pakistan", "parent": "asia"},
                },
                {
                    "data": {"id": "fr", "label": "France", "parent": "europe"},
                },
                {
                    "data": {"id": "eg", "label": "Egypt", "parent": "africa"},
                },
                # Children Nodes
                {
                    "data": {"id": "sh", "label": "Shanghai", "parent": "ch"},
                },
                {
                    "data": {"id": "bj", "label": "Beijing", "parent": "ch"},
                },
                {
                    "data": {"id": "ber", "label": "Berlin", "parent": "ger"},
                },
                {
                    "data": {"id": "ham", "label": "Hamburg", "parent": "ger"},
                },
                {
                    "data": {"id": "lah", "label": "Lahore", "parent": "pk"},
                },
                {
                    "data": {"id": "isl", "label": "Islamabad", "parent": "pk"},
                },
                {
                    "data": {"id": "par", "label": "Paris", "parent": "fr"},
                },
                {
                    "data": {"id": "nic", "label": "Nice", "parent": "fr"},
                },
                {
                    "data": {"id": "cai", "label": "Cairo", "parent": "eg"},
                },
                {
                    "data": {"id": "alex", "label": "Alexandria", "parent": "eg"},
                },
                # Edges
                {"data": {"source": "asia", "target": "europe"}, "classes": "countries"},
                {"data": {"source": "asia", "target": "africa"}, "classes": "countries"},
                {"data": {"source": "europe", "target": "africa"}, "classes": "countries"},
            ],
        )
    ]
)

if __name__ == "__main__":
    app.run(debug=True)

In [3]:
def create_cyto_elements(topics: Topics) -> dict:
    cyto_trees = {}
    for topic in topics.topics.values():
        cyto_elements = [{"data": {"id": topic.id, "label": topic.topic}}]
        for subtopic in topic.subtopics.values():
            cyto_elements.append(
                {"data": {"id": subtopic.id, "label": subtopic.subtopic}}
            )
            cyto_elements.append(
                {
                    "data": {
                        "source": topic.id,
                        "target": subtopic.id,
                        "label": "Subtopic",
                    }
                }
            )
            for concept in subtopic.concepts.values():
                cyto_elements.append(
                    {"data": {"id": concept.id, "label": concept.concept}}
                )
                cyto_elements.append(
                    {
                        "data": {
                            "source": subtopic.id,
                            "target": concept.id,
                            "label": "Concept",
                        }
                    }
                )
        cyto_trees[topic.id] = cyto_elements
    return cyto_trees

In [4]:
cyto_trees = create_cyto_elements(topics)

In [ ]:
cyto_tree = [{"id":}]

In [5]:
keys = ["Set Theory", "Complex Numbers"]

app = Dash(external_stylesheets=external_stylesheets)
cols = [
    dbc.Col(
        cyto.Cytoscape(
            id=key,
            elements=cyto_trees[key],
            layout={"name": "breadthfirst", "roots": f"[id = '{key}']"},
            stylesheet=[
                {
                    "selector": "node",
                    "style": {"background-color": "blue", "label": "data(label)"},
                },
                {
                    "selector": "edge",
                    "style": {"line-color": "black", "label": "data(label)"},
                },
                {
                    "selector": ".green",
                    "style": {"background-color": "green", "label:": "data(label)"},
                },
            ],
        )
    )
    for key in keys
]

app.layout = html.Div([dbc.Row(cols)])

for key in keys:
    other_key = keys[1] if key == keys[0] else keys[0]  # Get the other key

    @app.callback(
        Output(other_key, "stylesheet"),
        Input(key, "tapNodeData"),
    )
    def update_styles(tapNodeData):
        if tapNodeData is None:
            raise PreventUpdate
        else:
            new_styles = [
                {"selector": "node", "style": {"background-color": "blue"}},
                {
                    "selector": f'node[id = "{tapNodeData["id"]}"]',
                    "style": {"background-color": "green", "label": "data(label)"},
                },
            ]
            return new_styles

# Add a callback for each Cytoscape component
# for key in keys:

#     @app.callback(
#         Output(key, "stylesheet"),
#         Input(key, "tapNodeData"),
#     )
#     def update_styles(tapNodeData):
#         if tapNodeData is None:
#             raise PreventUpdate
#         else:
#             new_styles = [
#                 {"selector": "node", "style": {"background-color": "blue"}},
#                 {
#                     "selector": f'node[id = "{tapNodeData["id"]}"]',
#                     "style": {"background-color": "green", "label": "data(label)"},
#                 },
#             ]
#             return new_styles


if __name__ == "__main__":
    app.run_server(debug=True)